In [1]:
from module.engine.tensor import Tensor
from module.model.layers import Layer , Sequential
from module.model.loss import Loss
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

## Solving a regression problem with the sklearn diabetes dataset

In [19]:
#load data
X , y = datasets.load_diabetes(return_X_y=True, as_frame=False, scaled=True)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [20]:
model = Sequential([Layer(size=(10,15), activation='relu'),
                    Layer(size=(15,15) ,activation='relu'),
                    Layer(size=(15,5) ,activation='relu'),
                    Layer(size=(5,1))])
model.summary()

Layer(type)         Shape              Parameters
---------------------------------------------------
ReLULayer       Shape: (10, 15))      Params: 165
---------------------------------------------------
ReLULayer       Shape: (15, 15))      Params: 240
---------------------------------------------------
ReLULayer       Shape: (15, 5))      Params: 80
---------------------------------------------------
LinearLayer       Shape: (5, 1))      Params: 6
---------------------------------------------------
Trainable Parameters                          491


In [55]:
parameters = model.parameters()

In [58]:
niter = 500
lr = 0.01
bactch_size = 70
def train_model(split):
    lossi = [] 
    X, Y = {'train': (X_train, y_train), 'test': (X_test, y_test)}[split]
    for t in range(niter):
        size = np.random.randint(0, len(X), bactch_size)
        ypred = [model(X[x]) for x in size]
        loss = Loss.mean_squared_error(Y[size], ypred)
        lossi.append(loss.data)
        model.zero_grad()
        loss.backward()
        if t % 100 == 0:
            print(f'Epoch {t} loss: {loss.data}')
        for p in parameters:
            p.data = p.data - (lr * p.grad) 
    return lossi

In [59]:
pred = train_model('train')

Epoch 0 loss: 5905.823878129446
Epoch 100 loss: 5074.783827881712


In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 
n = Sequential([Layer(size=(3,4), activation='tanh'),
                    Layer(size=(4,4) ,activation='tanh'),
                    Layer(size=(4,1), activation='tanh')])
n.summary()
for k in range(200): 
    p = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, p))
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.01 * p.grad
    print(k, loss.data)